# Does the size of convolution kernels affect classifier performance? (Assuming parameter count is the same)

In [1]:
from __future__ import print_function
import keras

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import Bio
from Bio import SeqIO
import seaborn as sns
import pandas as pd
import Bio.motifs
%matplotlib inline
from sklearn import model_selection
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import scipy
sns.set_context('notebook')

In [3]:
# if not os.path.isdir('/home/jtao/analysis/genomic_grammar_analysis/'):
#     os.mkdir('/home/jtao/analysis/genomic_grammar_analysis')
# os.chdir('/home/jtao/analysis/genomic_grammar_analysis')
working_directory = '/home/jtao/analysis/genomic_grammar_analysis'

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from signal_models import *

In [6]:
from utility_functions import *

## Try different k-mer sizes

In [ ]:
all_rocs = []
all_accuracies = []
all_precisions = []
k_list = []
all_treatments = []
for ps in ['c57bl6_kla-1h_peaks.fasta', 'c57bl6_veh_peaks.fasta', 'c57bl6_il4-24h_peaks.fasta']:
    print(ps)
    positive_seqRecords = list(SeqIO.parse(working_directory + '/peak_sequences/' + ps, 'fasta'))
    negative_seqRecords = list(SeqIO.parse(working_directory + '/background_files/' + ps.replace('_peaks', '_background'), 'fasta'))[:len(positive_seqRecords)]

    fasta_seq = [str(x.seq[:200]) for x in positive_seqRecords] + [str(x[:200].seq) for x in negative_seqRecords]

    fasta_rc_seq = [str(x[:200].reverse_complement().seq) for x in positive_seqRecords] + \
        [str(x[:200].reverse_complement().seq) for x in negative_seqRecords]

    sequence_arrays = convert_sequences_to_array(fasta_seq)

    sequence_rc_arrays = convert_sequences_to_array(fasta_rc_seq)


    labels = [1 for x in positive_seqRecords] + [0 for x in negative_seqRecords]
    labels = np.array(labels)

    x_train, x_test, x_rc_train, x_rc_test, y_train, y_test = model_selection.train_test_split(sequence_arrays, sequence_rc_arrays, labels, test_size=0.2)

    num_classes = 2
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    for k in range(2,25):
        max_kmers = 4**k
        param_count=0
        kmers_to_use = 0
        while param_count <= 1552 - (4*k+1):
            kmers_to_use +=1
            param_count += (4*k+1)
        kmers_to_use = min(kmers_to_use, max_kmers)

        current_model = get_convolution_model(
            200,
            'classification',
            num_classes = 2,
            num_motifs = kmers_to_use,
            motif_size = k,
            num_dense_neurons = 32, 
            dropout_rate = 0.5)

        current_model.fit([x_train, x_rc_train], y_train,
              batch_size=64,
              epochs=10,
              verbose=1,
              validation_data=([x_test, x_rc_test], y_test))

        probs = current_model.predict([x_test, x_rc_test])

        roc = sklearn.metrics.roc_auc_score([y[1] for y in y_test], probs[:,1], )
        precision = sklearn.metrics.precision_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
        acc = sklearn.metrics.accuracy_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
        treatment = ps.split('_')[1]
        
        print(treatment, k, max_kmers, kmers_to_use, param_count)
        print(roc, precision, acc)

        all_rocs.append(roc)
        all_accuracies.append(acc)
        all_precisions.append(precision)
        k_list.append(k)
        all_treatments.append(treatment)

c57bl6_kla-1h_peaks.fasta


In [ ]:
frame = pd.DataFrame({'aucROC':all_rocs, 
                      'Accuracy':all_accuracies, 
                      'K':k_list, 
                      'Precision':all_precisions,
                      'Treatment':all_treatments})

In [ ]:
sns.factorplot(data = frame, x='K', y='aucROC', hue = 'Treatment', size=10)

In [ ]:
sns.factorplot(data = frame, x='K', y='Precision', hue = 'Treatment', size=10)

In [ ]:
sns.factorplot(data = frame, x='K', y='Accuracy', hue = 'Treatment', size=10)

## Try different motif counts

In [ ]:
all_rocs = []
all_accuracies = []
all_precisions = []
m_list = []
all_treatments = []
for ps in ['c57bl6_kla-1h_peaks.fasta', 'c57bl6_veh_peaks.fasta', 'c57bl6_il4-24h_peaks.fasta']:
    print(ps)
    positive_seqRecords = list(SeqIO.parse(working_directory + '/peak_sequences/' + ps, 'fasta'))
    negative_seqRecords = list(SeqIO.parse(working_directory + '/background_files/' + ps.replace('_peaks', '_background'), 'fasta'))[:len(positive_seqRecords)]

    fasta_seq = [str(x.seq[:200]) for x in positive_seqRecords] + [str(x[:200].seq) for x in negative_seqRecords]

    fasta_rc_seq = [str(x[:200].reverse_complement().seq) for x in positive_seqRecords] + \
        [str(x[:200].reverse_complement().seq) for x in negative_seqRecords]

    sequence_arrays = convert_sequences_to_array(fasta_seq)

    sequence_rc_arrays = convert_sequences_to_array(fasta_rc_seq)


    labels = [1 for x in positive_seqRecords] + [0 for x in negative_seqRecords]
    labels = np.array(labels)

    x_train, x_test, x_rc_train, x_rc_test, y_train, y_test = model_selection.train_test_split(sequence_arrays, sequence_rc_arrays, labels, test_size=0.2)

    num_classes = 2
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    for m in range(25,250, 25):
        current_model = get_convolution_model(
            200,
            'classification',
            num_classes = 2,
            num_motifs = m,
            motif_size = 10,
            num_dense_neurons = 32, 
            dropout_rate = 0.5)
        
        current_model.fit([x_train, x_rc_train], y_train,
              batch_size=128,
              epochs=10,
              verbose=1,
              validation_data=([x_test, x_rc_test], y_test))

        probs = current_model.predict([x_test, x_rc_test])

        roc = sklearn.metrics.roc_auc_score([y[1] for y in y_test], probs[:,1], )
        precision = sklearn.metrics.precision_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
        acc = sklearn.metrics.accuracy_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
        treatment = ps.split('_')[1]
        
        print(treatment, m, current_model.count_params())
        print(roc, precision, acc)

        all_rocs.append(roc)
        all_accuracies.append(acc)
        all_precisions.append(precision)
        m_list.append(m)
        all_treatments.append(treatment)

In [ ]:
frame = pd.DataFrame({'aucROC':all_rocs, 
                      'Accuracy':all_accuracies, 
                      'm':m_list, 
                      'Precision':all_precisions,
                      'Treatment':all_treatments})

sns.factorplot(data = frame, x='m', y='aucROC', hue = 'Treatment', size=10)
plt.ylim(0.7,1)
plt.show()
sns.factorplot(data = frame, x='m', y='Precision', hue = 'Treatment', size=10)
plt.ylim(0.7,1)
plt.show()

sns.factorplot(data = frame, x='m', y='Accuracy', hue = 'Treatment', size=10)
plt.ylim(0.7,1)
plt.show()


## Dense Only Analysis

In [ ]:
def get_dense_model(
    total_seq_length=200,
    seq_size = 150,
    num_classes = 2,
    neurons_per_layer=32
    motif_size = 20,
    num_dense_neurons = 25, 
    dropout_rate = 0.25
    ):
    input_fwd = Input(shape=(total_seq_length,4), name='input_fwd')
    input_rev = Input(shape=(total_seq_length,4), name='input_rev')

    # find motifs
    convolution_layer = Conv1D(filters=num_motifs, 
        kernel_size=motif_size,
        activation='relu',
        input_shape=(total_seq_length,4),
        name='convolution_layer',
        padding = 'same'
        )
    forward_motif_scores = convolution_layer(input_fwd)
    reverse_motif_scores = convolution_layer(input_rev)

    # crop motif scores to avoid parts of sequence where motif score is computed in only one direction
    to_crop = int((total_seq_length - seq_size)/2)
    crop_layer = Cropping1D(cropping=(to_crop, to_crop), 
        name='crop_layer')
    cropped_fwd_scores = crop_layer(forward_motif_scores)
    cropped_rev_scores = crop_layer(reverse_motif_scores)

    # calculate max scores for each orientation
    seq_pool_layer = MaxPool1D(pool_size=seq_size)
    max_fwd_scores = seq_pool_layer(cropped_fwd_scores)
    max_rev_scores = seq_pool_layer(cropped_rev_scores)

    # calculate max score for strand
    orientation_max_layer = Maximum()
    max_seq_scores = orientation_max_layer([max_fwd_scores, max_rev_scores])

    # fully connected layer
    dense_out = Dense(num_dense_neurons, activation='relu', 
                     )(max_seq_scores)

    # drop out
    drop_out = Dropout(0.25)(dense_out)

    # make prediction
    flattened = Flatten()(drop_out)
    predictions = Dense(num_classes,
                        activation = 'softmax', 
                       )(flattened)
    
    # define and compile model
    convolution_model = Model(inputs=[input_fwd, input_rev], outputs=predictions)

    return convolution_model

In [ ]:
for ps in ['c57bl6_kla-1h_peaks.fasta', 'c57bl6_veh_peaks.fasta', 'c57bl6_il4-24h_peaks.fasta']:
    print(ps)
    positive_seqRecords = list(SeqIO.parse('./peak_sequences/' + ps, 'fasta'))
    negative_seqRecords = list(SeqIO.parse('./background_files/' + ps.replace('_peaks', '_background'), 'fasta'))[:len(positive_seqRecords)]

    fasta_seq = [str(x.seq[:200]) for x in positive_seqRecords] + [str(x[:200].seq) for x in negative_seqRecords]

    fasta_rc_seq = [str(x[:200].reverse_complement().seq) for x in positive_seqRecords] + \
        [str(x[:200].reverse_complement().seq) for x in negative_seqRecords]

    sequence_arrays = convert_sequences_to_array(fasta_seq)

    sequence_rc_arrays = convert_sequences_to_array(fasta_rc_seq)


    labels = [1 for x in positive_seqRecords] + [0 for x in negative_seqRecords]
    labels = np.array(labels)

    x_train, x_test, x_rc_train, x_rc_test, y_train, y_test = model_selection.train_test_split(sequence_arrays, sequence_rc_arrays, labels, test_size=0.2)

    num_classes = 2
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    
    input_fwd = Input(shape=(total_seq_length,4), name='input_fwd')
    dense_out_1 = Dense(32, activation='relu', 
                 )(input_fwd)
    dense_out_2 = Dense(32, activation='relu', 
                 )(dense_out_1)
    dense_out_3 = Dense(32, activation='relu', 
                 )(dense_out_2)
    # drop out
    drop_out = Dropout(0.5)(dense_out_3)

    # make prediction
    flattened = Flatten()(drop_out)
    predictions = Dense(num_classes,
                        activation = 'softmax', 
                       )(flattened)
    
    current_model = Model(inputs=[input_fwd], outputs=predictions)


    current_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    current_model.fit([x_train], y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=([x_test], y_test))

    probs = current_model.predict([x_test])

    roc = sklearn.metrics.roc_auc_score([y[1] for y in y_test], probs[:,1], )
    precision = sklearn.metrics.precision_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
    acc = sklearn.metrics.accuracy_score([y[1] for y in y_test], [1 if x > 0.5 else 0 for x in probs[:,1]])
    treatment = ps.split('_')[1]

    print(treatment )
    print(roc, precision, acc)
